In [1]:
import pandas as pd
#import numpy as np
import tools
import itertools

In [2]:
# exibe todas as colunas do dataframe
pd.set_option('display.max_columns', None)
#pd.set_option('display.max_rows', None)

# avaliacao

In [63]:
if 'df_avaliacao_db' in locals():
    del(df_avaliacao_db)

date_columns = ['DT_CALCULO', 'DT_REAJUSTE_BENEFICIO']
df_avaliacao_db = pd.read_csv("input_csv/avaliacao.csv", sep=";", decimal=",", parse_dates=date_columns)

In [64]:
if 'df_avaliacao_tmp'in locals():
    del(df_avaliacao_tmp)
    
name_columns = {'ID_AVALIACAO': 'avaliacao', 'ID_CADASTRO': 'cadastro', 'IDPLANOPREV': 'plano_previdencia', 'ID_PLANO_BENEFICIO': 'plano_beneficio', 'DT_CALCULO': 'data_calculo',
                 'DT_REAJUSTE_BENEFICIO': 'data_reajuste_beneficio', 'VL_BENEFICIO_MINIMO': 'beneficio_minimo', 'PC_DESPESA_ADM_PARTICIPANTE': 'despesa_administ_partic',
                 'PC_DESPESA_ADM_PATROCINADORA': 'despesa_administ_patroc', 'PC_SAIDA_BPD': 'saida_bpd', 'PC_SAIDA_PORTABILIDADE': 'saida_portabilidade', 'PC_SAIDA_RESGATE': 'saida_resgate',
                 'VL_PECULIO_MINIMO_MORTE': 'peculio_minimo_morte', 'PC_PECULIO_MINIMO_MORTE': 'fator_peculio_morte_ativo', 'VL_PECULIO_MORTE_ASSISTIDO': 'fator_peculio_morte_assistido',
                 'NR_PAGTOS_BENEF_CONTRIB_ANO': 'numero_contribuicoes_ano', 'PC_FAIXA_01_CONTRIBUICAO': 'faixa_1_contribuicao', 'PC_FAIXA_02_CONTRIBUICAO': 'faixa_2_contribuicao',
                 'PC_FAIXA_03_CONTRIBUICAO': 'faixa_3_contribuicao', 'PC_META_CUSTEIO_ADMIN': 'taxa_carregamento_admin', 'PC_TAXA_ADMIN_BENEFICIO': 'taxa_admin_beneficio',
                 'PC_TAXA_REAL_CRESC_SALARIAL': 'taxa_cresc_salarial', 'PC_TAXA_REAL_CRESC_BENEFICIO': 'taxa_cresc_beneficio', 'PC_FATOR_VLR_REAL_SALARIO': 'fator_capacidade_salario',
                 'PC_FATOR_VLR_REAL_BEN_FUNCEF': 'fator_capacidade_funcef', 'PC_FATOR_VLR_REAL_BEN_INSS': 'fator_capacidade_inss', 'PC_FATOR_REAJ_BENEF_LIQUIDO': 'fator_capacidade_beneficio',
                 'PC_OPCAO_BUA': 'opcao_bua', 'PC_SAQUE_BUA': 'saque_bua'}
drop_columns = ['DS_PLANO_BENEFICIO', 'FL_CALCULAR_DEMONS_RESULTADO', 'FL_FLUXO_RECEITA_DESPESA_FOLHA', 'FL_MEMORIA_CALCULO', 'CD_COMPOSICAO_FAMILIAR']

df_avaliacao_tmp = df_avaliacao_db.rename(columns = name_columns).drop(drop_columns, axis=1)

In [65]:
df_avaliacao_tmp.despesa_administ_partic = df_avaliacao_tmp.despesa_administ_partic.map(tools.convertToPercent)
df_avaliacao_tmp.despesa_administ_patroc = df_avaliacao_tmp.despesa_administ_patroc.map(tools.convertToPercent)
df_avaliacao_tmp.saida_bpd = df_avaliacao_tmp.saida_bpd.map(tools.convertToPercent)
df_avaliacao_tmp.saida_portabilidade = df_avaliacao_tmp.saida_portabilidade.map(tools.convertToPercent)
df_avaliacao_tmp.saida_resgate = df_avaliacao_tmp.saida_resgate.map(tools.convertToPercent)
df_avaliacao_tmp.faixa_1_contribuicao = df_avaliacao_tmp.faixa_1_contribuicao.map(tools.convertToPercent)
df_avaliacao_tmp.faixa_2_contribuicao = df_avaliacao_tmp.faixa_2_contribuicao.map(tools.convertToPercent)
df_avaliacao_tmp.faixa_3_contribuicao = df_avaliacao_tmp.faixa_3_contribuicao.map(tools.convertToPercent)
df_avaliacao_tmp.taxa_carregamento_admin = df_avaliacao_tmp.taxa_carregamento_admin.map(tools.convertToPercent)
df_avaliacao_tmp.taxa_admin_beneficio = df_avaliacao_tmp.taxa_admin_beneficio.map(tools.convertToPercent)
df_avaliacao_tmp.taxa_cresc_salarial = df_avaliacao_tmp.taxa_cresc_salarial.map(tools.convertToPercent)
df_avaliacao_tmp.taxa_cresc_beneficio = df_avaliacao_tmp.taxa_cresc_beneficio.map(tools.convertToPercent)
df_avaliacao_tmp.fator_capacidade_salario = df_avaliacao_tmp.fator_capacidade_salario.map(tools.convertToPercent)
df_avaliacao_tmp.fator_capacidade_funcef = df_avaliacao_tmp.fator_capacidade_funcef.map(tools.convertToPercent)
df_avaliacao_tmp.fator_capacidade_inss = df_avaliacao_tmp.fator_capacidade_inss.map(tools.convertToPercent)
df_avaliacao_tmp.fator_capacidade_beneficio = df_avaliacao_tmp.fator_capacidade_beneficio.map(tools.convertToIndice)
df_avaliacao_tmp.opcao_bua = df_avaliacao_tmp.opcao_bua.map(tools.convertToPercent)
df_avaliacao_tmp.saque_bua = df_avaliacao_tmp.saque_bua.map(tools.convertToPercent)

# premissas do plano

In [73]:
# df_prem_plano_db = None
if 'df_prem_plano_db' in locals():
    del(df_prem_plano_db)
    
df_prem_plano_db = pd.read_csv("input_csv/premissa_planos.csv", sep=";", decimal=",")

In [74]:
if 'df_prem_plano_tmp' in locals():
    del(df_prem_plano_tmp)
    
name_columns = {'ID_PLANO_BENEFICIO': 'plano_beneficio', 'IDADE_INICIO_CONTRIB_INSS_MAS': 'idade_inicio_contrib_inss_mas', 'IDADE_INICIO_CONTRIB_INSS_FEM': 'idade_inicio_contrib_inss_fem',
                'IDADE_APOSENT_FUNDACAO_MAS': 'idade_aposent_fundacao_mas', 'IDADE_APOSENT_FUNDACAO_FEM': 'idade_aposent_fundacao_fem', 'TEMPO_CONTRIB_INSS_MAS': 'tempo_contrib_inss_mas',
                'TEMPO_CONTRIB_INSS_FEM': 'tempo_contrib_inss_fem', 'MAIORIDADE_PLANO': 'maioridade_plano', 'PERCENTUAL_SRB': 'percentual_srb', 'TEMPO_CARENCIA_APOSENTADORIA': 'carencia_aposentadoria'}
drop_columns = ['ID_PREMISSA_PLANO']
# df_prem_plano = None
df_prem_plano_tmp = df_prem_plano_db.rename(columns = name_columns).drop(drop_columns, axis=1)
df_prem_plano_tmp.percentual_srb = df_prem_plano_tmp.percentual_srb.map(tools.convertToPercent)

# premissas globais

In [78]:
#df_prem_global_db = None
if 'df_prem_global_db' in locals():
    del(df_prem_global_db)
    
date_columns = ['DT_ORIGEM_BNH', 'DT_LEI_9876_1999', 'DT_MEDIA_80_MAIORES_SALARIOS']
df_prem_global_db = pd.read_csv("input_csv/premissa_global.csv", sep=";", decimal=",", parse_dates = date_columns)

In [79]:
if 'df_prem_global_tmp' in locals():
    del(df_prem_global_tmp)
    
name_columns = {'DT_ORIGEM_BNH': 'data_bnh', 'DT_LEI_9876_1999': 'data_lei_9876', 'DT_MEDIA_80_MAIORES_SALARIOS': 'data_media_maiores_salarios', 'TETO_CONTRIBUICAO_INSS': 'teto_contribuicao_inss',
                'TETO_BENEFICIO_INSS': 'teto_beneficio_inss', 'SALARIO_MINIMO': 'salario_minimo', 'COTA_FAMILIAR_PENSAO': 'cota_pensao_familiar',
                'PROB_APOSENTADO_CASADO_MAS': 'prob_aposentado_casado_mas', 'PROB_APOSENTADO_CASADO_FEM': 'prob_aposentado_casado_fem', 'DIF_IDADE_CONJUGE_MAS': 'dif_idade_conjuge_mas',
                'DIF_IDADE_CONJUGE_FEM': 'dif_idade_conjuge_fem', 'LX_INICIAL': 'lx_inicial'}
drop_columns = ['ID_PREMISSA_GLOBAL']
#df_prem_global = None
df_prem_global_tmp = df_prem_global_db.rename(columns = name_columns).drop(drop_columns, axis=1)
df_prem_global_tmp.cota_pensao_familiar = df_prem_global_tmp.cota_pensao_familiar.map(tools.convertToPercent)
df_prem_global_tmp.prob_aposentado_casado_mas = df_prem_global_tmp.prob_aposentado_casado_mas.map(tools.convertToPercent)
df_prem_global_tmp.prob_aposentado_casado_fem = df_prem_global_tmp.prob_aposentado_casado_fem.map(tools.convertToPercent)
df_prem_global_tmp['avaliacao'] = df_avaliacao_tmp.avaliacao

# taxa de juros

In [80]:
# df_taxa_juros_db = None
if 'df_taxa_juros_db' in locals():
    del(df_taxa_juros_db)
    
df_taxa_juros_db = pd.read_csv("input_csv/taxa_juros.csv", sep=";", decimal=",")

In [81]:
if 'df_taxa_juros' in locals():
    del(df_taxa_juros)
    
name_columns = {'NR_TEMPO_TAXA_JUROS': 'tempo', 'VL_TAXA_JUROS': 'taxa_juros'}
# df_taxa_juros = None
df_taxa_juros = df_taxa_juros_db.rename(columns= name_columns)
df_taxa_juros.taxa_juros = df_taxa_juros.taxa_juros.map(tools.convertToPercent)

# taxa de risco

In [82]:
# df_taxa_risco_db = None
if 'df_taxa_risco_db' in locals():
    del(df_taxa_risco_db)
    
df_taxa_risco_db = pd.read_csv("input_csv/taxa_risco.csv", sep=";", decimal=",")

In [83]:
if 'df_taxa_risco' in locals():
    del(df_taxa_risco)
    
name_columns = {'NR_TEMPO_TAXA_RISCO': 'tempo', 'ID_RESPONSABILIDADE': 'responsabilidade', 'VL_TAXA_RISCO': 'taxa_risco'}
# df_taxa_risco = None
df_taxa_risco = df_taxa_risco_db.rename(columns = name_columns)
df_taxa_risco.taxa_risco = df_taxa_risco.taxa_risco.map(tools.convertToPercent)

# indice INPC - cotacao moeda

In [84]:
# df_indice_db = None
if 'df_indice_db' in locals():
    del(df_indice_db)
    
date_columns = ['COTDATA']
df_indice_db = pd.read_csv("input_csv/cotacao.csv", sep=";", decimal=",", parse_dates = date_columns)

In [85]:
if 'df_indice' in locals():
    del(df_indice)
    
name_columns = {'COTDATA': 'data', 'COTVALOR': 'valor'}
# df_indice = None
df_indice = df_indice_db.rename(columns= name_columns)
df_indice.valor = df_indice.valor.map(tools.convertToIndice)

# indexador monetario

In [86]:
# df_indexador_db = None
if 'df_indexador_db' in locals():
    del(df_indexador_db)
    
df_indexador_db = pd.read_csv("input_csv/indexador_monetario.csv", sep=";", decimal=",")

In [87]:
if 'df_indexador' in locals():
    del(df_indexador)
    
name_columns = {'ID_REFERENCIA_IDX_MON': 'indexador', 'PC_REAJUSTE': 'valor'}
drop_columns = ['DS_REFERENCIA_IDX_MON']
# df_indexador = None
df_indexador = df_indexador_db.rename(columns=name_columns).drop(drop_columns, axis=1)

# reajuste salarial

In [88]:
# df_reaj_salarial_db = None
if 'df_reaj_salarial_db' in locals():
    del(df_reaj_salarial_db)
    
df_reaj_salarial_db = pd.read_csv("input_csv/reajuste_salarial.csv", sep=";", decimal=",")

In [89]:
if 'df_reaj_salarial' in locals():
    del(df_reaj_salarial)
    
name_columns = {'CD_TIPO_REAJUSTE_SALARIAL': 'tipo', 'ID_PATROCINADORA': 'patrocinadora', 'PC_REAJUSTE': 'valor'}
drop_columns = ['DS_TIPO_REAJUSTE_SALARIAL']
# df_reaj_salarial = None
df_reaj_salarial = df_reaj_salarial_db.rename(columns = name_columns).drop(drop_columns, axis= 1)
df_reaj_salarial.valor = df_reaj_salarial.valor.map(tools.convertToPercent)

# join : avaliacao + premissa plano + premissa global

In [90]:
if 'df_avaliacao' in locals():
    del(df_avaliacao)
    
df_avaliacao = pd.merge(df_avaliacao_tmp, df_prem_plano_tmp, how='inner', on='plano_beneficio')

In [91]:
df_avaliacao = pd.merge(df_avaliacao, df_prem_global_tmp, how='inner', on='avaliacao')

In [92]:
df_avaliacao.dtypes

avaliacao                                 int64
cadastro                                  int64
plano_previdencia                         int64
plano_beneficio                           int64
data_calculo                     datetime64[ns]
data_reajuste_beneficio          datetime64[ns]
beneficio_minimo                        float64
despesa_administ_partic                 float64
despesa_administ_patroc                 float64
saida_bpd                               float64
saida_portabilidade                     float64
saida_resgate                           float64
peculio_minimo_morte                    float64
fator_peculio_morte_ativo               float64
fator_peculio_morte_assistido           float64
numero_contribuicoes_ano                  int64
faixa_1_contribuicao                    float64
faixa_2_contribuicao                    float64
faixa_3_contribuicao                    float64
taxa_carregamento_admin                 float64
taxa_admin_beneficio                    

In [93]:
df_avaliacao

,avaliacao,cadastro,plano_previdencia,plano_beneficio,data_calculo,data_reajuste_beneficio,beneficio_minimo,despesa_administ_partic,despesa_administ_patroc,saida_bpd,saida_portabilidade,saida_resgate,peculio_minimo_morte,fator_peculio_morte_ativo,fator_peculio_morte_assistido,numero_contribuicoes_ano,faixa_1_contribuicao,faixa_2_contribuicao,faixa_3_contribuicao,taxa_carregamento_admin,taxa_admin_beneficio,taxa_cresc_salarial,taxa_cresc_beneficio,fator_capacidade_salario,fator_capacidade_funcef,fator_capacidade_inss,fator_capacidade_beneficio,opcao_bua,saque_bua,idade_inicio_contrib_inss_mas,idade_inicio_contrib_inss_fem,idade_aposent_fundacao_mas,idade_aposent_fundacao_fem,tempo_contrib_inss_mas,tempo_contrib_inss_fem,maioridade_plano,percentual_srb,carencia_aposentadoria,data_bnh,data_lei_9876,data_media_maiores_salarios,teto_contribuicao_inss,teto_beneficio_inss,salario_minimo,cota_pensao_familiar,prob_aposentado_casado_mas,prob_aposentado_casado_fem,dif_idade_conjuge_mas,dif_idade_conjuge_fem,lx_inicial
0,25445,10809,74,5,2016-12-31,2016-01-01,0.0,0.0398,0.0398,0.0,0.5,0.5,0.0,2.5,11.62,13,0.0,0.0,0.0,0.0398,0.009,0.0239,0.0,0.98,0.98,0.98,2.2177,0.0,0.0,18,18,53,48,35,30,24,0.2,10,1998-09-23,1999-11-29,1999-07-31,5189.82,5078.94,880.0,0.8,0.85,0.6,5,2,10000000


# participante

In [95]:
if 'df_partic_db' in locals():
    del(df_partic_db)

date_columns = ['DT_NASCIMENTO', 'DT_OPCAO_BPD', 'DT_ADMISSAO', 'DT_ASSOCIACAO_FUNDACAO']
data_type = {'NR_MATRICULA': str, 'matricula_titular': str}

df_partic_db = pd.read_csv("input_csv/participante.csv", sep=";", decimal=",", encoding="latin1", parse_dates = date_columns, dtype = data_type, low_memory = False)

In [98]:
if 'df_partic_temp' in locals():
    del(df_partic_temp)
    
name_columns = {'ID_PARTICIPANTE': 'id_participante', 'NR_MATRICULA': 'matricula', 'DT_NASCIMENTO': 'data_nascimento_partic', 'IR_SEXO': 'sexo_partic', 'ID_PATROCINADORA': 'patrocinadora',
                'CD_ESTADO_CIVIL': 'estado_civil', 'DT_ADMISSAO': 'data_admissao', 'DT_ASSOCIACAO_FUNDACAO': 'data_associacao', 'PC_BENEFICIO_ESPECIAL': 'pbe', 'FL_DEFICIENTE': 'deficiente',
                'NR_MATRICULA_TITULAR': 'matricula_titular', 'FL_MIGRADO': 'migrado'}
drop_columns = ['CD_SITUACAO_PATROC', 'DS_ESTADO_CIVIL', 'DT_OPCAO_BPD']
# df_partic2 = df_partic1.rename(columns= nomes_colunas).drop(['CD_SITUACAO_PATROC', 'DS_ESTADO_CIVIL', 'DT_OPCAO_BPD'], axis= 1)
df_partic_temp = df_partic_db.rename(columns= name_columns).drop(drop_columns, axis= 1)

In [99]:
df_partic_temp.deficiente = list(map(tools.convertToBoolean, df_partic_temp.deficiente))
df_partic_temp.migrado = df_partic_temp.migrado.map(tools.convertToBoolean)
df_partic_temp.pbe = df_partic_temp.pbe.map(tools.convertToPercent)

# plano de beneficios

In [100]:
if 'df_plano_db' in locals():
    del(df_plano_db)
    
date_columns = ['DT_ADESAO']
df_plano_db = pd.read_csv("input_csv/plano_beneficio.csv", sep=";", decimal=",", parse_dates = date_columns)

In [101]:
if 'df_plano_tmp' in locals():
    del(df_plano_tmp)
    
name_columns = {'ID_PARTICIPANTE': 'id_participante', 'DT_ADESAO': 'data_adesao', 'VL_SLD_SUBCONTA_PARTICIPANTE': 'saldo_conta_partic', 'VL_SLD_SUBCONTA_PATROCINADORA': 'saldo_conta_patroc',
                'VL_RESERVA_BPD': 'reserva_bpd', 'VL_SALDO_PORTADO': 'saldo_portado', 'VL_BEN_SALDADO_INICIAL': 'beneficio_saldado', 'VL_SALARIO_PARTICIPACAO': 'salario_participacao',
                'PC_CONTRIBUICAO_PARTICIPANTE': 'contribuicao_partic', 'PC_CONTRIBUICAO_PATROCINADORA': 'contribuicao_patroc'}
df_plano_tmp = df_plano_db.rename(columns= name_columns)

In [102]:
df_plano_tmp.contribuicao_partic = df_plano_tmp.contribuicao_partic.map(tools.convertToPercent)
df_plano_tmp.contribuicao_patroc = df_plano_tmp.contribuicao_patroc.map(tools.convertToPercent)

# beneficio inss

In [103]:
if 'df_benef_inss_db' in locals():
    del(df_benef_inss_db)

date_columns = ['DT_INICIO_BENEFICIO']
df_benef_inss_db = pd.read_csv("input_csv/beneficio_inss.csv", sep=";", decimal=",", parse_dates = date_columns)

In [104]:
if 'df_benef_inss_tmp' in locals():
    del(df_benef_inss_tmp)
    
name_columns = {'ID_PARTICIPANTE': 'id_participante', 'DT_INICIO_BENEFICIO': 'dib_inss', 'VL_VALOR': 'valor_beneficio_inss'}
df_benef_inss_tmp = df_benef_inss_db.rename(columns = name_columns)

# beneficio previdencia

In [105]:
if 'df_benef_prev_db' in locals():
    del(df_benef_prev_db)

date_columns = ['DT_INICIO_BENEFICIO']
df_benef_prev_db = pd.read_csv("input_csv/beneficio_funcef.csv", sep=";", decimal=",", encoding="latin1", parse_dates= date_columns)

In [106]:
if 'df_benef_prev_tmp' in locals():
    del(df_benef_prev_tmp)
    
name_columns = {'ID_PARTICIPANTE': 'id_participante', 'IDBENEFICIO': 'beneficio', 'VL_VALOR': 'valor_beneficio_funcef', 'DT_INICIO_BENEFICIO': 'dib_funcef'}
drop_columns = ['NOME']
df_benef_prev_tmp = df_benef_prev_db.rename(columns= name_columns).drop(drop_columns, axis= 1)

In [107]:
df_benef_prev_tmp['tipo_beneficio'] = df_benef_prev_tmp.beneficio.map(tools.convertToTipoBeneficio)
df_benef_prev_tmp = df_benef_prev_tmp[df_benef_prev_tmp.tipo_beneficio != 0]
drop_columns = ['beneficio']
df_benef_prev_tmp = df_benef_prev_tmp.drop(drop_columns, axis = 1)

# dependentes

In [108]:
if 'df_depend_db' in locals():
    del(df_depend_db)
    
date_columns = ['DT_NASCIMENTO']
df_depend_db = pd.read_csv("input_csv/dependente.csv", sep=";", decimal=",", encoding="latin1", parse_dates= date_columns)

In [109]:
if 'df_depend_tmp' in locals():
    del(df_depend_tmp)

# nomes_colunas = {'ID_PARTICIPANTE': 'id_participante', 'CD_GRAU_DEPENDENCIA': 'parentesco', 'DT_NASCIMENTO': 'data_nascimento', 'IR_SEXO': 'sexo', 'FL_INVALIDO': 'invalido'}
name_columns = {'ID_PARTICIPANTE': 'id_participante', 'CD_GRAU_DEPENDENCIA': 'parentesco', 'FL_INVALIDO': 'invalido'}
drop_columns = ['FL_DESIGNADO_RESGATE', 'FL_DEPENDENTE_LEGAL']
df_depend_tmp = df_depend_db.rename(columns= name_columns).drop(drop_columns, axis=1)

In [110]:
df_depend_tmp.invalido = df_depend_tmp.invalido.map(tools.convertToBoolean)

# dependente valido (conjuge)

In [112]:
if 'df_depend_valido_tmp' in locals():
    del(df_depend_valido_tmp)
    
df_depend_valido_tmp = df_depend_tmp[(df_depend_tmp.parentesco == 'COM') & (df_depend_tmp.invalido == False)]

name_columns = {'IR_SEXO': 'sexo_depend_valido', 'DT_NASCIMENTO': 'data_nascimento_depend_valido'}
drop_columns = ['parentesco', 'invalido']

df_depend_valido_tmp = df_depend_valido_tmp.rename(columns = name_columns).drop(drop_columns, axis = 1)

# dependente temporario (filho menor idade valido)

In [113]:
if 'df_depend_temporario_tmp' in locals():
    del(df_depend_temporario_tmp)
    
df_depend_temporario_tmp = df_depend_tmp[(df_depend_tmp.parentesco == 'FIL') & (df_depend_tmp.invalido == False)]

name_columns = {'IR_SEXO': 'sexo_depend_temporario', 'DT_NASCIMENTO': 'data_nascimento_depend_temporario'}
drop_columns = ['parentesco', 'invalido']

df_depend_temporario_tmp = df_depend_temporario_tmp.rename(columns = name_columns).drop(drop_columns, axis = 1)

# dependente invalido (conjuge invalido ou filho invalido)

In [115]:
if 'df_depend_invalido_tmp' in locals():
    del(df_depend_invalido_tmp)

df_depend_invalido_tmp = df_depend_tmp[df_depend_tmp.invalido == True]

name_columns = {'IR_SEXO': 'sexo_depend_invalido', 'DT_NASCIMENTO': 'data_nascimento_depend_invalido'}
drop_columns = ['parentesco', 'invalido']

df_depend_invalido_tmp = df_depend_invalido_tmp.rename(columns = name_columns).drop(drop_columns, axis = 1)

# join: participante + plano

In [121]:
if 'df_participante' in locals():
    del(df_participante)
    
if len(df_partic_temp) == len(df_plano_tmp):
    df_participante = pd.merge(df_partic_temp, df_plano_tmp, how='inner', on='id_participante')

In [123]:
if 'df_participante' in locals():
    if len(df_participante) == len(df_partic_temp):
        df_participante = pd.merge(df_participante, df_benef_inss_tmp, how='left', on='id_participante')
        df_participante = pd.merge(df_participante, df_benef_prev_tmp, how='left', on='id_participante')
        df_participante = pd.merge(df_participante, df_depend_valido_tmp, how='left', on='id_participante')
        df_participante = pd.merge(df_participante, df_depend_temporario_tmp, how='left', on='id_participante')
        df_participante = pd.merge(df_participante, df_depend_invalido_tmp, how='left', on='id_participante')

In [125]:
df_participante.dtypes

id_participante                               int64
matricula                                    object
data_nascimento_partic               datetime64[ns]
sexo_partic                                  object
patrocinadora                                 int64
estado_civil                                 object
data_admissao                        datetime64[ns]
data_associacao                      datetime64[ns]
pbe                                         float64
deficiente                                     bool
matricula_titular                            object
migrado                                        bool
data_adesao                          datetime64[ns]
saldo_conta_partic                          float64
saldo_conta_patroc                          float64
reserva_bpd                                 float64
saldo_portado                               float64
beneficio_saldado                           float64
salario_participacao                        float64
CD_SITUACAO_

In [126]:
df_participante.head()

,id_participante,matricula,data_nascimento_partic,sexo_partic,patrocinadora,estado_civil,data_admissao,data_associacao,pbe,deficiente,matricula_titular,migrado,data_adesao,saldo_conta_partic,saldo_conta_patroc,reserva_bpd,saldo_portado,beneficio_saldado,salario_participacao,CD_SITUACAO_FUNDACAO,contribuicao_partic,contribuicao_patroc,dib_inss,valor_beneficio_inss,valor_beneficio_funcef,dib_funcef,tipo_beneficio,data_nascimento_depend_valido,sexo_depend_valido,data_nascimento_depend_temporario,sexo_depend_temporario,data_nascimento_depend_invalido,sexo_depend_invalido
0,10036696,0000017,1958-04-09,M,91008,C,1983-02-21,1983-02-21,0.0,False,NaN,False,2006-01-09,131686.86,129553.83,0.0,0.0,1925.46,13574.57,1,0.1200,0.1200,NaT,NaN,NaN,NaT,NaN,1960-09-15,F,1994-08-19,M,NaT,NaN
1,10036697,0000030,1961-04-02,F,91008,C,1983-10-02,1983-10-02,0.0,False,NaN,False,2006-01-09,144031.70,138926.83,0.0,0.0,4109.00,21851.89,1,0.0536,0.0536,NaT,NaN,NaN,NaT,NaN,NaT,NaN,NaT,NaN,NaT,NaN
2,10036698,0000129,1959-12-17,M,91008,C,1983-02-16,1983-02-16,0.0,False,NaN,False,2006-01-09,124043.99,121493.24,0.0,0.0,3519.18,13432.95,1,0.1200,0.1200,NaT,NaN,NaN,NaT,NaN,1962-12-19,F,1993-05-04,M,NaT,NaN
3,10036699,0000135,1958-02-09,M,91008,C,1983-02-16,1983-02-16,0.0,False,NaN,False,2006-01-09,152490.91,149994.91,0.0,0.0,2915.02,13448.68,1,0.1200,0.1200,NaT,NaN,NaN,NaT,NaN,1962-06-10,F,1997-03-07,M,NaT,NaN
4,10036700,0000170,1959-04-11,M,91008,D,1983-02-16,1983-02-16,0.0,False,NaN,False,2006-01-09,145427.77,142346.31,0.0,0.0,2742.64,15057.81,1,0.0500,0.0500,NaT,NaN,NaN,NaT,NaN,1981-11-04,F,2007-05-21,M,NaT,NaN


In [127]:
df_participante.tail()

,id_participante,matricula,data_nascimento_partic,sexo_partic,patrocinadora,estado_civil,data_admissao,data_associacao,pbe,deficiente,matricula_titular,migrado,data_adesao,saldo_conta_partic,saldo_conta_patroc,reserva_bpd,saldo_portado,beneficio_saldado,salario_participacao,CD_SITUACAO_FUNDACAO,contribuicao_partic,contribuicao_patroc,dib_inss,valor_beneficio_inss,valor_beneficio_funcef,dib_funcef,tipo_beneficio,data_nascimento_depend_valido,sexo_depend_valido,data_nascimento_depend_temporario,sexo_depend_temporario,data_nascimento_depend_invalido,sexo_depend_invalido
91505,10128201,9776825,2003-04-20,F,91008,S,2000-04-12,2007-05-09,0.0,False,0546710,False,2007-05-09,0.0,0.0,0.0,0.0,0.0,0.0,26,0.0,0.0,NaT,NaN,47.90,2016-03-11,4.0,NaT,NaN,NaT,NaN,NaT,NaN
91506,10128202,9776831,2005-05-10,F,91008,S,2000-04-12,2007-05-09,0.0,False,0546710,False,2007-05-09,0.0,0.0,0.0,0.0,0.0,0.0,26,0.0,0.0,NaT,NaN,47.90,2016-03-11,4.0,NaT,NaN,NaT,NaN,NaT,NaN
91507,10128203,9776860,2001-07-03,F,91008,S,2002-02-05,2002-01-06,0.0,False,0642661,False,2012-09-04,0.0,0.0,0.0,0.0,0.0,0.0,26,0.0,0.0,NaT,NaN,3955.97,2016-10-23,4.0,NaT,NaN,NaT,NaN,NaT,NaN
91508,10128204,9776908,2001-08-14,M,91008,S,1989-08-05,1989-08-05,0.0,False,0269630,True,2006-01-09,0.0,0.0,0.0,0.0,0.0,0.0,26,0.0,0.0,2016-09-27,5029.02,3126.90,2016-09-27,4.0,NaT,NaN,NaT,NaN,NaT,NaN
91509,10128205,1128459,1967-06-05,F,91008,S,1979-08-20,1979-08-20,0.0,False,3348703,True,2006-01-09,0.0,0.0,0.0,0.0,0.0,0.0,25,0.0,0.0,NaT,NaN,453.24,2016-11-30,4.0,NaT,NaN,NaT,NaN,NaT,NaN


In [ ]:
#df_participantes.valor_beneficio_inss.map(lambda x: if pd.isnull(x)) = [0.00 for x in df_participantes.valor_beneficio_inss if pd.isnull(x)]

In [ ]:
[0.00 for x in df_participantes.valor_beneficio_inss if np.isnan(x)]

In [ ]:
df_participantes[pd.isnull(df_participantes['valor_beneficio_funcef'])].valor_beneficio_funcef = 0.00

In [ ]:
#if len(df_depend_valido) > 0:
    #df_participantes['idade_depend_valido'] = list(map(tools.calculateAge, df_depend_valido.data_nascimento, itertools.repeat(df_avaliacao.data_calculo, len(df_depend_valido))))
    #df_depend_valido.head(10)
#else:
    #del(df_depend_valido)
    
#df_depend_valido = df_depend_valido.drop(['data_nascimento'], axis = 1)

In [ ]:
#if len(df_depend_invalido) > 0:
    #df_participantes['idade_invalido'] = list(map(tools.calculateAge, df_depend_invalido.data_nascimento_valido, itertools.repeat(df_avaliacao.data_calculo, len(df_depend_invalido))))
    #df_depend_invalido.head(10)
#else:
#    del(df_depend_invalido)

#df_depend_invalido = df_depend_invalido.drop(['data_nascimento'], axis = 1)

In [ ]:
#if len(df_depend_temporario) > 0:
    #df_depend_temporario['idade_temporario'] = list(map(tools.calculateAge, df_depend_temporario.data_nascimento, itertools.repeat(df_avaliacao.data_calculo, len(df_depend_temporario))))
    #df_depend_temporario.head(10)
#else:
#    del(df_depend_temporario)

#df_depend_temporario = df_depend_temporario.drop(['data_nascimento'], axis = 1)